In [33]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 140)

In [34]:
health_timeline = pd.read_excel(
    "health_timeline_intelligence.xlsx"
)

In [35]:
rpm_simulated = pd.read_excel(
    "daily_patient_monitoring_analytics.xlsx",
    sheet_name="daily_patient_monitoring_analytics"
)

In [36]:
rpm_simulated.rename(columns={
    "simulated_ecg_risk_flag": "rpm_ecg_risk_flag",
    "simulated_ecg_reason": "rpm_ecg_reason"
}, inplace=True)

In [37]:
rpm_cols = [
    "date",
    "cardio_stress_score",
    "risk_level",
    "fatigue_risk_flag",
    "anomaly_flag",
    "ecg_available",
    "rpm_ecg_risk_flag",
    "rpm_ecg_reason"
]

rpm_reference = rpm_simulated[rpm_cols].copy()

In [38]:
rpm_reference.head()

,date,cardio_stress_score,risk_level,fatigue_risk_flag,anomaly_flag,ecg_available,rpm_ecg_risk_flag,rpm_ecg_reason
0,2021-01-01,0.34,Low,0,0,False,0,NaN
1,2021-01-02,0.26,Low,0,0,False,0,NaN
2,2021-01-03,0.33,Low,0,0,False,0,NaN
3,2021-01-04,0.21,Low,0,0,False,0,NaN
4,2021-01-05,0.25,Low,0,0,False,0,NaN


In [39]:
health_timeline["date"].dtype
rpm_reference["date"].dtype

dtype('<M8[ns]')

In [40]:
health_timeline.shape

(1390, 36)

In [41]:
rpm_reference.shape

(1476, 8)

In [42]:
validation_view = health_timeline.merge(
    rpm_reference,
    on="date",
    how="left"
)

In [43]:
validation_view.shape

(1405, 43)

In [44]:
validation_view.head(3)

,date,sleep_available,steps_available,hr_available,total_sleep_min,sleep_efficiency,deep_pct,rem_pct,fragmented_sleep_flag,sleep_extreme_flag,stage_available_flag,total_steps,intervals,low_activity_flag,high_activity_flag,false_spm_flag,avg_hr,min_hr,max_hr,hr_std,high_hr_minutes,low_hr_minutes,elevated_hr_flag,high_hr_variability_flag,health_risk_state,daily_risk_score,risk_score,early_warning_flag,risk_streak_days,elevated_risk_streak_days,risk_days_last_14,risk_escalation_flag,final_intervention_tier,intervention_recommendation,risk_trajectory_state,clinical_summary,cardio_stress_score,risk_level,fatigue_risk_flag,anomaly_flag,ecg_available,rpm_ecg_risk_flag,rpm_ecg_reason
0,2021-10-06,1,1,0,215.883333,0.420853,0.0,0.0,1.0,0.0,0.0,6884.0,41.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mild Risk,1,30,0,1,0,NaN,0,Lifestyle Adjustment,Improve sleep schedule and duration; Reduce ni...,Stable,Health parameters appear stable.,0.20,Low,0.0,0.0,False,0.0,NaN
1,2021-10-07,1,1,0,248.950000,0.861468,0.0,0.0,0.0,0.0,0.0,8421.0,66.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mild Risk,1,30,0,2,0,NaN,0,Lifestyle Adjustment,Improve sleep schedule and duration,Stable,Health parameters appear stable.,0.30,Low,0.0,0.0,False,0.0,NaN
2,2021-10-08,0,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12032.0,82.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Stable,0,0,0,0,0,NaN,0,Stable,Reduce physical overexertion,Stable,Health parameters appear stable.,0.24,Low,0.0,0.0,False,0.0,NaN


In [45]:
validation_view["ecg_supports_ai_risk"] = (
    (validation_view["risk_escalation_flag"] == 1) &
    (validation_view["rpm_ecg_risk_flag"] == 1)
).astype(int)

In [46]:
validation_view["ai_only_risk"] = (
    (validation_view["risk_escalation_flag"] == 1) &
    ((validation_view["rpm_ecg_risk_flag"].isna()) |
     (validation_view["rpm_ecg_risk_flag"] == 0))
).astype(int)

In [47]:
validation_view["rpm_only_risk"] = (
    (validation_view["risk_escalation_flag"] == 0) &
    (validation_view["rpm_ecg_risk_flag"] == 1)
).astype(int)

In [48]:
validation_view[
    ["ecg_supports_ai_risk", "ai_only_risk", "rpm_only_risk"]
].sum()

ecg_supports_ai_risk     15
ai_only_risk            880
rpm_only_risk             0
dtype: int64

In [49]:
escalated_days = validation_view[
    validation_view["risk_escalation_flag"] == 1
]

escalated_days[
    ["rpm_ecg_risk_flag", "ecg_supports_ai_risk"]
].value_counts()

rpm_ecg_risk_flag  ecg_supports_ai_risk
0.0                0                       578
1.0                1                        15
Name: count, dtype: int64

In [50]:
def validation_conclusion(row):
    if row["ecg_supports_ai_risk"] == 1:
        return "ECG findings reinforce AI-detected risk"
    if row["ai_only_risk"] == 1:
        return "AI detected early risk without ECG confirmation"
    if row["rpm_only_risk"] == 1:
        return "ECG anomaly detected without broader physiological risk"
    return "No significant risk signals detected"

In [51]:
validation_view["ecg_validation_summary"] = (
    validation_view.apply(validation_conclusion, axis=1)
)

In [52]:
validation_view[
    validation_view["ecg_supports_ai_risk"] == 1
][
    ["date", "risk_score", "final_intervention_tier",
     "rpm_ecg_risk_flag", "ecg_validation_summary"]
].head(5)

,date,risk_score,final_intervention_tier,rpm_ecg_risk_flag,ecg_validation_summary
78,2022-02-05,30,Clinical Attention Recommended,1.0,ECG findings reinforce AI-detected risk
80,2022-02-06,0,Clinical Attention Recommended,1.0,ECG findings reinforce AI-detected risk
82,2022-02-07,0,Clinical Attention Recommended,1.0,ECG findings reinforce AI-detected risk
84,2022-02-08,0,Clinical Attention Recommended,1.0,ECG findings reinforce AI-detected risk
86,2022-02-09,0,Clinical Attention Recommended,1.0,ECG findings reinforce AI-detected risk


In [55]:
sorted(validation_view.columns.tolist())

['ai_only_risk',
 'anomaly_flag',
 'avg_hr',
 'cardio_stress_score',
 'clinical_summary',
 'daily_risk_score',
 'date',
 'deep_pct',
 'early_warning_flag',
 'ecg_available',
 'ecg_supports_ai_risk',
 'ecg_validation_summary',
 'elevated_hr_flag',
 'elevated_risk_streak_days',
 'false_spm_flag',
 'fatigue_risk_flag',
 'final_intervention_tier',
 'fragmented_sleep_flag',
 'health_risk_state',
 'high_activity_flag',
 'high_hr_minutes',
 'high_hr_variability_flag',
 'hr_available',
 'hr_std',
 'intervals',
 'intervention_recommendation',
 'low_activity_flag',
 'low_hr_minutes',
 'max_hr',
 'min_hr',
 'rem_pct',
 'risk_days_last_14',
 'risk_escalation_flag',
 'risk_level',
 'risk_score',
 'risk_streak_days',
 'risk_trajectory_state',
 'rpm_ecg_reason',
 'rpm_ecg_risk_flag',
 'rpm_only_risk',
 'sleep_available',
 'sleep_efficiency',
 'sleep_extreme_flag',
 'stage_available_flag',
 'steps_available',
 'total_sleep_min',
 'total_steps']

In [56]:
sleep_cols = [
    "sleep_extreme_flag",
    "fragmented_sleep_flag"
]

validation_view["sleep_risk_evidence"] = (
    validation_view[sleep_cols].fillna(0).max(axis=1)
)

In [57]:
activity_cols = [
    "high_activity_flag",
    "low_activity_flag",
    "false_spm_flag"
]

validation_view["activity_risk_evidence"] = (
    validation_view[activity_cols].fillna(0).max(axis=1)
)

In [58]:
hr_cols = [
    "elevated_hr_flag",
    "high_hr_variability_flag",
    "high_hr_minutes",
    "low_hr_minutes"
]

validation_view["hr_risk_evidence"] = (
    validation_view[hr_cols].fillna(0).max(axis=1)
)

In [59]:
validation_view[
    ["sleep_risk_evidence", "activity_risk_evidence", "hr_risk_evidence"]
].sum()

sleep_risk_evidence         180.0
activity_risk_evidence     1063.0
hr_risk_evidence          50240.0
dtype: float64

In [60]:
ecg_days = validation_view[
    validation_view["rpm_ecg_risk_flag"] == 1
]

ecg_days[[
    "date",
    "risk_score",
    "health_risk_state",
    "final_intervention_tier",
    "sleep_risk_evidence",
    "activity_risk_evidence",
    "hr_risk_evidence",
    "ecg_validation_summary"
]]

,date,risk_score,health_risk_state,final_intervention_tier,sleep_risk_evidence,activity_risk_evidence,hr_risk_evidence,ecg_validation_summary
78,2022-02-05,30,Mild Risk,Clinical Attention Recommended,0.0,1.0,0.0,ECG findings reinforce AI-detected risk
80,2022-02-06,0,Stable,Clinical Attention Recommended,0.0,0.0,0.0,ECG findings reinforce AI-detected risk
82,2022-02-07,0,Stable,Clinical Attention Recommended,0.0,0.0,0.0,ECG findings reinforce AI-detected risk
84,2022-02-08,0,Stable,Clinical Attention Recommended,0.0,0.0,0.0,ECG findings reinforce AI-detected risk
86,2022-02-09,0,Stable,Clinical Attention Recommended,0.0,0.0,0.0,ECG findings reinforce AI-detected risk
389,2023-04-01,0,Stable,Clinical Attention Recommended,0.0,1.0,0.0,ECG findings reinforce AI-detected risk
391,2023-04-02,20,Mild Risk,Clinical Attention Recommended,0.0,1.0,0.0,ECG findings reinforce AI-detected risk
393,2023-04-03,0,Stable,Clinical Attention Recommended,0.0,0.0,0.0,ECG findings reinforce AI-detected risk
395,2023-04-04,0,Stable,Clinical Attention Recommended,0.0,1.0,0.0,ECG findings reinforce AI-detected risk
397,2023-04-05,0,Stable,Clinical Attention Recommended,0.0,1.0,0.0,ECG findings reinforce AI-detected risk


In [61]:
validation_view["hr_risk_evidence"] = (
    (validation_view["elevated_hr_flag"] == 1) |
    (validation_view["high_hr_variability_flag"] == 1) |
    (validation_view["high_hr_minutes"].fillna(0) > 0) |
    (validation_view["low_hr_minutes"].fillna(0) > 0)
).astype(int)


In [62]:
validation_view["hr_risk_evidence"].value_counts()

hr_risk_evidence
0    968
1    437
Name: count, dtype: int64

In [63]:
ecg_days = validation_view[
    validation_view["rpm_ecg_risk_flag"] == 1
]

ecg_days[[
    "date",
    "risk_score",
    "health_risk_state",
    "final_intervention_tier",
    "sleep_risk_evidence",
    "activity_risk_evidence",
    "hr_risk_evidence",
    "ecg_validation_summary"
]]


,date,risk_score,health_risk_state,final_intervention_tier,sleep_risk_evidence,activity_risk_evidence,hr_risk_evidence,ecg_validation_summary
78,2022-02-05,30,Mild Risk,Clinical Attention Recommended,0.0,1.0,0,ECG findings reinforce AI-detected risk
80,2022-02-06,0,Stable,Clinical Attention Recommended,0.0,0.0,0,ECG findings reinforce AI-detected risk
82,2022-02-07,0,Stable,Clinical Attention Recommended,0.0,0.0,0,ECG findings reinforce AI-detected risk
84,2022-02-08,0,Stable,Clinical Attention Recommended,0.0,0.0,0,ECG findings reinforce AI-detected risk
86,2022-02-09,0,Stable,Clinical Attention Recommended,0.0,0.0,0,ECG findings reinforce AI-detected risk
389,2023-04-01,0,Stable,Clinical Attention Recommended,0.0,1.0,0,ECG findings reinforce AI-detected risk
391,2023-04-02,20,Mild Risk,Clinical Attention Recommended,0.0,1.0,0,ECG findings reinforce AI-detected risk
393,2023-04-03,0,Stable,Clinical Attention Recommended,0.0,0.0,0,ECG findings reinforce AI-detected risk
395,2023-04-04,0,Stable,Clinical Attention Recommended,0.0,1.0,0,ECG findings reinforce AI-detected risk
397,2023-04-05,0,Stable,Clinical Attention Recommended,0.0,1.0,0,ECG findings reinforce AI-detected risk
